In [1]:
import yaml
from omegaconf import OmegaConf
from yolo.module.model_module import ModelModule
from yolo.module.data_module import DataModule

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers

from yolo.module.model.yolox.utils.boxes import postprocess
from yolo.utils.eval.evaluation import to_coco_format, evaluation

## データ、モデルの用意

In [5]:
yaml_file = "./../config/notebook.yaml"
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)
config = OmegaConf.create(config)

data = DataModule(config)
model = ModelModule(config)

In [ ]:
model.model.eval()

In [ ]:
train_dataset = data.train_dataset
val_dataset = data.valid_dataset

In [ ]:
index = 0
img, label, info, id = train_dataset[index]

In [ ]:
preds = model(img, None)
processed_pred = postprocess(prediction=preds,
                            num_classes=config.model.head.num_classes,
                            conf_thre=config.model.postprocess.conf_thre,
                            nms_thre=config.model.postprocess.nms_thre)

if config.dataset.name == "coco":
    from ..yolo.module.data.dataset.coco.coco_classes import COCO_CLASSES as CLASSES
elif config.dataset.name == "dsec":
    from ..yolo.module.data.dataset.dsec.label import CLASSES
            
classes = CLASSES
categories = [{"id": idx + 1, "name": name} for idx, name in enumerate(classes)]
num_data = len(label)
gt, pred = to_coco_format(gts=label, detections=processed_pred, categories=categories, height=640, width=640)

# COCO evaluationでスコアを取得
scores = evaluation(Gt=gt, Dt=pred, num_data=num_data)
